# MODULOS

In [ ]:
import os   
import shutil
from datetime import date
import pandas as pd
import psycopg2
import glob
import time

import textwrap ## fecha
import numpy as np
from datetime import date ## fecha actual
from datetime import datetime

from time import sleep ## parar codigo

import subprocess

from sqlalchemy import create_engine
import xlsxwriter

from xlsxwriter.utility import xl_rowcol_to_cell
from pandas.errors import ParserError
# import oracledb
from sqlalchemy import text ## modulo para exportar información geográfica a sql

import pandas as pd

ruta_destino = r'D:\ExcelPostgress'

# Detalles de la conexión a la base de datos PostgreSQL
db_user = 'postgres'
db_password = '12345'
db_host = 'localhost'
db_port = '5432'
db_name = 'Antioquia2024'

# Crear un motor de conexión usando SQLAlchemy
#engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
engine2 = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

## Se crea conexión a la BD
con2 = psycopg2.connect(user=db_user, password=db_password, host=db_host, port=db_port, database=db_name)

# Cursor para ejecutar las querys
cur2 = con2.cursor()



## FUNCIONES

In [ ]:
def get_col_widths(dataframe):
    widths = []
    for column in dataframe.columns:
        # Asegurarse de que dataframe[column] es una Series
        if isinstance(dataframe[column], pd.Series):
            column_width = max(dataframe[column].astype(str).map(len).max(), len(column)) + 0.5
        else:
            column_width = len(column) + 0.5
        widths.append(column_width)
    return widths

# Función de Estilos de Pandas:
def estilo_pandas(df,writer,sheetname): 
    # Obtener el índice del DataFrame antes de realizar cambios
    df_index = df.index
    # Llenar los valores NaN con una cadena vacía
    df = df.fillna('')

    # Suponiendo que 'writer' es el objeto Workbook y 'df' es tu DataFrame
    workbook = writer.book
    worksheet = writer.sheets[sheetname[:31]]
            
    # Definir formato para celdas de encabezado con bordes blancos
    cell_format_header = workbook.add_format({
        'bg_color': '#1c2b52',
        'font_color': '#ffffff',
        'bold': True,
        'align': 'center',  # Centrar texto horizontalmente
        'valign': 'vcenter',  # Centrar texto verticalmente
        'top': 1,   # Borde superior
        'bottom': 1,   # Borde inferior
        'left': 1,   # Borde izquierdo
        'right': 1   # Borde derecho
    })
    # Iterar sobre las columnas y aplicar el formato a las celdas de la primera fila
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, cell_format_header)
    # Ajustar el tamaño de letra y agregar bordes a todas las celdas del DataFrame
    for idx, row in df.iterrows():
        for col_num, value in enumerate(row):
            worksheet.write(idx, col_num, value, workbook.add_format({'font_size': 10, 'top': 1, 'bottom': 1, 'left': 1, 'right': 1}))
    # Ajustar el ancho de las columnas automáticamente según el contenido
    for i, width in enumerate(get_col_widths(df)):
        worksheet.set_column(i, i, width) 
    # Restablecer el índice del DataFrame
    df.index = df_index

# Función de Estilos de Pandas FILA ENCABEZADOS:
def estilo_pandas_encabezado(df,writer,sheetname): 
    # Obtener el índice del DataFrame antes de realizar cambios
    df_index = df.index
    # Llenar los valores NaN con una cadena vacía
    df = df.fillna('')
    # Suponiendo que 'writer' es el objeto Workbook y 'df' es tu DataFrame
    workbook = writer.book
    worksheet = writer.sheets[sheetname[:31]]
    # Definir formato para celdas de encabezado con bordes blancos
    cell_format_header = workbook.add_format({
        'bg_color': '#1c2b52',
        'font_color': '#ffffff',
        'bold': True,
        'align': 'center',  # Centrar texto horizontalmente
        'valign': 'vcenter',  # Centrar texto verticalmente
        'top': 1,   # Borde superior
        'bottom': 1,   # Borde inferior
        'left': 1,   # Borde izquierdo
        'right': 1   # Borde derecho
    })
    # Iterar sobre las columnas y aplicar el formato a las celdas de la primera fila
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, cell_format_header)
    # Ajustar el ancho de las columnas automáticamente según el contenido
    for i, width in enumerate(get_col_widths(df)):
        worksheet.set_column(i, i, width) 
    # Restablecer el índice del DataFrame
    df.index = df_index

In [ ]:

# Obtener la lista de esquemas desde information_schema
esquemas = pd.read_sql("""
    SELECT schema_name
    FROM information_schema.schemata
    WHERE schema_name NOT IN ('pg_catalog', 'information_schema', 'pg_toast')
""", con=engine2)

# Crear un diccionario para almacenar tablas por esquema
esquema_tablas = {}

# Iterar sobre cada esquema para listar sus tablas
for esquema in esquemas['schema_name']:
    tablas = pd.read_sql(f"""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = '{esquema}'
        AND table_type = 'BASE TABLE'
    """, con=engine2)
    
    esquema_tablas[esquema] = tablas['table_name'].tolist()

# Diccionario con cantidad de elementos
diccionario = []

# Exportar cada tabla a un archivo Excel
for esquema, tablas in esquema_tablas.items():
    print(f"\n Export para el Esquema {esquema} \n")
    # Ruta de almacenamiento de los excels
    ruta = os.path.join(ruta_destino, esquema)
    if not os.path.exists(ruta):
        os.makedirs(ruta)
        print(f"Se crea la carpeta {esquema}, en {ruta_destino}")
    
    for tabla in tablas:
        writer = pd.ExcelWriter(os.path.join(ruta,f"{tabla}.xlsx"), engine='xlsxwriter')

        # Leer los datos de la tabla
        df = pd.read_sql(f"SELECT * FROM {esquema}.{tabla}", con=engine2)
        df = df.reset_index(drop=True) 
        df.index += 1
        df.insert(0, "ID", df.index)
        df.to_excel(writer, sheet_name=tabla[:31], index=False)
        estilo_pandas_encabezado(df,writer,tabla)
        writer.close()

        print(f"Tabla {tabla} exportada a {ruta}")

        diccionario.append({
            "XTF": esquema,
            "COD_MUN": esquema[1:6],
            "Tabla": tabla,
            "N° Registros": len(df)
        })

diccionario = pd.DataFrame(diccionario)
writer2 = pd.ExcelWriter(os.path.join(ruta_destino,f"_Consolidado_descarga.xlsx"), engine='xlsxwriter')

diccionario = diccionario.reset_index(drop=True) 
diccionario.index += 1
diccionario.insert(0, "ID", diccionario.index)
diccionario.to_excel(writer2, sheet_name="_Consolidado_descarga"[:31], index=False)
estilo_pandas_encabezado(diccionario,writer2,"_Consolidado_descarga"[:31])

lc_predio = diccionario[diccionario["Tabla"] == "lc_predio"].reset_index(drop=True)
lc_predio.to_excel(writer2, sheet_name="lc_predio"[:31], index=False)
estilo_pandas_encabezado(lc_predio,writer2,"lc_predio"[:31])



writer2.close()




 Export para el Esquema pg_temp_1 

Se crea la carpeta pg_temp_1, en D:\ExcelPostgress

 Export para el Esquema pg_toast_temp_1 

Se crea la carpeta pg_toast_temp_1, en D:\ExcelPostgress

 Export para el Esquema public 

Se crea la carpeta public, en D:\ExcelPostgress
Tabla spatial_ref_sys exportada a D:\ExcelPostgress\public

 Export para el Esquema olaya 

Se crea la carpeta olaya, en D:\ExcelPostgress
Tabla gm_geometry2dlistvalue exportada a D:\ExcelPostgress\olaya
Tabla cc_centropoblado exportada a D:\ExcelPostgress\olaya
Tabla cc_corregimiento exportada a D:\ExcelPostgress\olaya
Tabla cc_limitemunicipio exportada a D:\ExcelPostgress\olaya
Tabla cc_manzana exportada a D:\ExcelPostgress\olaya
Tabla cc_perimetrourbano exportada a D:\ExcelPostgress\olaya
Tabla cc_sectorrural exportada a D:\ExcelPostgress\olaya
Tabla cc_sectorurbano exportada a D:\ExcelPostgress\olaya
Tabla cc_vereda exportada a D:\ExcelPostgress\olaya
Tabla cc_localidadcomuna exportada a D:\ExcelPostgress\olaya
Tabla